In [3]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from CrispyLLM_RoPE.modeling_crispy import CrispyLLMConfig, CrispyForCausalLM
import shutil

# 1. Model Sınıfı Kayıt (Auto ile kullanabilmek için)

In [ ]:
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)

# 2. Tokenizer Yükleme

In [ ]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("hosmankarabulut/Crispy-2.8B-CLM")

# 3. Modelin Tasarlanması

In [ ]:
crispy_config = CrispyLLMConfig(
    attn_implementation="flash_attention_2",   # 🔁 Attention tipi: FlashAttention v2 kullanılacak
    decoder_dropout=0.1 ,                      # 🧠 Decoder block'lardaki dropout oranı (Feedforward + Attention sonrası dropout)
    attention_dropout=0.0,                     # 🔄 Attention bloklarındaki dropout oranı (sadece attention matrisine uygulanır)
    use_flash_attention_2=True,                # ⚡ FlashAttention2 modülünün kullanılacağını belirten özel flag (model içi kullanım için)
    vocab_size=len(tokenizer.get_vocab()),     # 🔠 Tokenizer'dan alınan toplam kelime (token) sayısı → lm_head çıkış boyutu
    n_heads=30,                                # 🧩 Her katmanda kaç tane attention head kullanılacağı
    max_position_embeddings=2048,              # Modelin gömebileceği maksimum pozisyon sayısı
    max_seq_len=1024,                          # Eğitim sırasında kullandığın seqlen (chunk boyutu)
    hidden_size=64*30,                         # 📐 Gizli katman boyutu → hidden_size = n_heads * head_dim (burada 64 * 30 = 1920)
    num_hidden_layers=30,                      # 🏗 Transformer blok sayısı (katman sayısı)
    dtype="bfloat16",                          # 🧮 Hesaplamalarda kullanılacak veri tipi (bfloat16 → TPU / A100 uyumlu)
    pad_token_id = tokenizer.pad_token_id ,    # 🧊 Padding token ID'si (genellikle 0)
    bos_token_id = tokenizer.bos_token_id,     # 🚀 Metin başlangıç token ID'si (örneğin: "<s>" veya "<|bos|>")
    eos_token_id = tokenizer.eos_token_id,     # 🛑 Metin bitiş token ID'si (örneğin: "</s>" veya "<|eos|>")
    unk_token_id = tokenizer.unk_token_id,     # ❓ Bilinmeyen (unknown) token ID'si (örneğin: "<unk>")
   
    auto_map= {
    "AutoConfig": "modeling_crispy.CrispyLLMConfig",
    "AutoModelForCausalLM": "modeling_crispy.CrispyForCausalLM"
    }, 
)

crispy_config._attn_implementation_autoset = True  # 👈 Buraya ekliyorsun

model = AutoModelForCausalLM.from_config(crispy_config)

('../CrispyTest/CrispyLLM-RoPE/tokenizer_config.json',
 '../CrispyTest/CrispyLLM-RoPE/special_tokens_map.json',
 '../CrispyTest/CrispyLLM-RoPE/tokenizer.json')

# 4. Model İnceleme

In [5]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(50000, 1920)
    )
  )
  (decoderBlocks): ModuleList(
    (0-29): 30 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1920, out_features=5760, bias=True)
        (o_proj): Linear(in_features=1920, out_features=1920, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1920,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1920, out_features=7680, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=7680, out_features=3840, bias=True)
          (linear2): Linear(in_features=1920, out_features=7680, bias=True)
        )
        (ln2): Linear(in_features=7680, out_features=1920, bias=True)
      )
    

# 5. Kayıt işlemi

In [ ]:
model.save_pretrained("../CrispyTest/CrispyLLM-RoPE")
tokenizer.save_pretrained("../CrispyTest/CrispyLLM-RoPE") 

In [ ]:
shutil.copy("./CrispyLLM_RoPE/modeling_crispy.py", "../CrispyTest/CrispyLLM-RoPE/modeling_crispy.py")

# 6. Model Testi

In [6]:
import torch

text = "Selam"

# 1️⃣ Tokenizer → Tensor
inputs = tokenizer(text, return_tensors="pt")

# 2️⃣ Modelin cihazına gönder (örneğin CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}
model = model.to(device)

# 3️⃣ Inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
print("✅ Logits shape:", logits.shape)


✅ Logits shape: torch.Size([1, 4, 50000])


# 7. Parametre Hesaplama

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🔢 Toplam Parametre: {total_params:,}")
print(f"🧠 Eğitilebilir (trainable): {trainable_params:,}")

# 8. Model Huggingface Yükleme

In [10]:
# Upload klasörü
from huggingface_hub import upload_folder
from huggingface_hub import HfApi

# Repo oluşturulmamışsa önce oluştur
HfApi().create_repo(repo_id="Crispy-2.8B-CLM", private=True)


upload_folder(
    repo_id="hosmankarabulut/Crispy-2.8B-CLM",  # kendi kullanıcı adını yaz
    folder_path="../CrispyTest/CrispyLLM-RoPE",
    commit_message="🚀 Custom modeling_crispy.py ile model yüklendi",
)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/725M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hosmankarabulut/Crispy-2.8B-CLM/commit/ecdab5b492b664188e9090cc57aa5fdaf32c1c02', commit_message='🚀 Custom modeling_crispy.py ile model yüklendi', commit_description='', oid='ecdab5b492b664188e9090cc57aa5fdaf32c1c02', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hosmankarabulut/Crispy-2.8B-CLM', endpoint='https://huggingface.co', repo_type='model', repo_id='hosmankarabulut/Crispy-2.8B-CLM'), pr_revision=None, pr_num=None)